In [1]:
from openai import OpenAI

openai_client = OpenAI()

In [7]:
from gitsource import GithubRepositoryDataReader, chunk_documents

reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

ModuleNotFoundError: No module named 'gitsource'

In [ ]:
from minsearch import AppendableIndex

In [ ]:
index = AppendableIndex(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

In [ ]:
def search(query):
    results = index.search(
        query=query,
        num_results=5
    )
    return results

In [ ]:
import json

RAG_INSTRUCTIONS = """
You're a documentation assistant. Answer the QUESTION based on the CONTEXT from our documentation.

Use only facts from the CONTEXT when answering.
If the answer isn't in the CONTEXT, say so.
"""

RAG_PROMPT_TEMPLATE = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(question, search_results):
    context = json.dumps(search_results, indent=2)
    return RAG_PROMPT_TEMPLATE.format(
        question=question,
        context=context
    )

In [ ]:
question = "How do I create a dahsbord in Evidently?"
search_results = search(question)
user_prompt = build_prompt(question, search_results)

In [ ]:
messages = [
    {"role": "system", "content": RAG_INSTRUCTIONS},
    {"role": "user", "content": user_prompt}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
)

In [ ]:
print(response.output_text)

The provided context does not contain specific instructions on how to create a dashboard in Evidently. If you need guidance on creating a dashboard, please refer to additional documentation or resources related to Evidently.


In [ ]:
instructions = """
You're a documentation assistant. 

Answer the user question using the documentation knowledge base

Use only facts from the knowledge base when answering.
IMPORTANT: f you cannot find the answer, inform the user.
"""

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the documentation database for relevant results based on a query string.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to look up in the index"
            }
        },
        "required": [
            "query"
        ]
    }
}


In [ ]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
)

response.usage.input_tokens

63

In [ ]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    tools=[search_tool],
)

response.usage.input_tokens

110

In [ ]:
tool_call = response.output[0]
tool_call

ResponseFunctionToolCall(arguments='{"query":"create dashboard in Evidently"}', call_id='call_sMYwgDoVKK2nCZdje7nsev3j', name='search', type='function_call', id='fc_021d8079b9203008006989b84ddbcc819f8989f48ceb60e936', status='completed')

In [ ]:
messages.append(tool_call)

In [ ]:
tool_call.arguments

'{"query":"create dashboard in Evidently"}'

In [ ]:
arguments = json.loads(tool_call.arguments)
arguments

{'query': 'create dashboard in Evidently'}

In [ ]:
search_results = search(query='create dashboard in Evidently')

In [ ]:
**arguments

In [ ]:
search_results = search(**arguments)
search_results[:1]

[{'start': 0,
  'content': 'Dashboards let you create Panels to visualize evaluation results over time. Note that to be able to populate the panels, you must first add Reports with evaluation results to the Project.\n\n<Check>\n  No-code Dashboards are available in the Evidently Cloud and Enterprise.\n</Check>\n\n## Adding Tabs\n\nBy default, new Panels appear on a single Dashboard. You can add multiple Tabs to organize them.\n\n**To add a Tab**:\n\n- Enter "Edit" mode on the Dashboard (top right corner).\n- Click the plus sign with "add Tab" on the left.\n- To create a custom Tab, select "empty" and enter a name.\n\nTo simplify setup, you can start with pre-built Tabs. These are dashboard templates with preset Panel combinations:\n\n![Add Dashboard Tab](/images/dashboard/add_dashboard_tab_v2.gif)\n\n**Pre-built Tabs** rely on having related Metrics (or Presets that include the specific Metrics) within the Project. If the necessary data is not available, the Panels will appear empty un

In [ ]:
call_output = {
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": json.dumps(search_results),
}


In [ ]:
messages.append(call_output)
messages

[{'role': 'system',
  'content': "\nYou're a documentation assistant. \n\nAnswer the user question using the documentation knowledge base\n\nUse only facts from the knowledge base when answering.\nIMPORTANT: f you cannot find the answer, inform the user.\n"},
 {'role': 'user', 'content': 'How do I create a dahsbord in Evidently?'},
 ResponseFunctionToolCall(arguments='{"query":"create dashboard in Evidently"}', call_id='call_sMYwgDoVKK2nCZdje7nsev3j', name='search', type='function_call', id='fc_021d8079b9203008006989b84ddbcc819f8989f48ceb60e936', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_sMYwgDoVKK2nCZdje7nsev3j',
  'output': '[{"start": 0, "content": "Dashboards let you create Panels to visualize evaluation results over time. Note that to be able to populate the panels, you must first add Reports with evaluation results to the Project.\\n\\n<Check>\\n  No-code Dashboards are available in the Evidently Cloud and Enterprise.\\n</Check>\\n\\n## Adding Tabs

In [ ]:
response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=messages,
    tools=[search_tool],
)

response.usage.input_tokens

4075

In [ ]:
print(response.output_text)

To create a dashboard in Evidently, follow these steps:

### Step 1: Set Up Your Project
1. **Connect to Evidently Cloud** and **create a Project**. This is necessary before you can manage dashboards.

### Step 2: Create a Dashboard
- **Add Tabs**: By default, new panels appear on a single dashboard. You can add multiple tabs to organize them.
  - **To add a Tab**:
    - Enter "Edit" mode on the Dashboard (top right corner).
    - Click the plus sign with "add Tab".
    - Choose "empty" to create a custom Tab and enter a name.

### Step 3: Add Panels
- You can add various types of panels like text panels, counters, pie charts, bar plots, and line plots.
- **To add a Panel**:
  1. Enter "Edit" mode on the Dashboard.
  2. Click on the "Add Panel" button.
  3. Follow the prompts to configure the panel.
  4. Review your setup using the preview.
  5. Click "Save" and select the Tab where you want to add the Panel.

### Example of Adding a Panel
You may add panels using Python API as follows

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field


class RAGResponse(BaseModel):
    """
    This model provides a structured answer with metadata about the response,
    including confidence, categorization, and follow-up suggestions.
    """

    answer: str = Field(description="The main answer to the user's question in markdown")
    found_answer: bool = Field(description="True if relevant information was found in the documentation")
    confidence: float = Field(description="Confidence score from 0.0 to 1.0 indicating how certain the answer is")
    confidence_explanation: str = Field(description="Explanation about the confidence level")
    answer_type: Literal["how-to", "explanation", "troubleshooting", "comparison", "reference"] = Field(description="The category of the answer")
    followup_questions: list[str] = Field(description="Suggested follow-up questions the user might want to ask")


In [ ]:
response = openai_client.responses.parse(
    model='gpt-4o-mini',
    input=messages,
    tools=[search_tool],
    text_format=RAGResponse
)

response.usage.input_tokens

4299

In [ ]:
4299 - 4075

224

In [ ]:
rag_response = response.output_parsed
print(rag_response.answer)

### Creating a Dashboard in Evidently

To create a dashboard in Evidently, follow these steps:

1. **Enter Edit Mode**: 
   - Look for the "Edit" mode button at the top right corner of the Dashboard interface. Click it to begin customizing your dashboard.

2. **Add Tabs (Optional)**: 
   - By default, panels appear on a single dashboard. You can add multiple tabs to organize your panels by clicking the plus sign labeled "add Tab" on the left. You can customize the tab name or choose from pre-built tabs based on related metrics.

3. **Add Panels**: 
   - To add different types of panels (e.g., text panels, counters, pie charts, etc.):
     - Click on the **"Add Panel"** button next to the dashboard.
     - Follow the prompts to configure the panel, including selecting metrics and adjusting display options like type and aggregation.
     - Use the preview feature to check your setup before saving.
     - Select the Tab where the panel should be added and click "Save".

4. **Metrics Selec